In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

In [26]:
# Import Data
oecd_healthcare = pd.read_csv('../data/OECD_healthcare_coverage.csv')
hypertension_adults = pd.read_csv('../data/hypertension_adults.csv')
hypertension_female = pd.read_csv('../data/hypertension_female.csv')
hypertension_male = pd.read_csv('../data/hypertension_male.csv')
overweight_adults = pd.read_csv('../data/overweight_adults.csv')
overweight_female = pd.read_csv('../data/overweight_female.csv')
overweight_male = pd.read_csv('../data/overweight_male.csv')
oop_spend_25pct = pd.read_csv('../data/spending_25_oop.csv')
urban_pop = pd.read_csv('../data/urban_population.csv')
urbpop_as_pct_total = pd.read_csv('../data/urban_pop_pct_tot.csv')
petersen_kff_data = pd.read_csv('../data/petersen_KFF_data.csv')
continents = pd.read_csv('../data/continents.csv')
diabetes_adults = pd.read_csv('../data/IDF_total_adult20to79_in_1000s.csv')
diabetes_deaths = pd.read_csv('../data/IDF_deaths_diabetes.csv')
ihme_heart_failure_prevalence = pd.read_csv('../data/IHME_heart_failure_prevalence.csv')
ihme_heart_failure_number = pd.read_csv('../data/IHME_heart_failure_number.csv')

## Hypertension Adults Analysis

In [3]:
# Generate the list of column names for the years 1990 to 2023
year_columns = [f'{year} [YR{year}]' for year in range(1990, 2020)]
new_col_names = {col: int(col.split(' ')[0]) for col in year_columns} # Create a dictionary to rename the columns

# Filter the DataFrame for the specific Series Name and select the year columns
filtered_adult_hypertension = hypertension_adults[
    hypertension_adults['Series Name'] == "Prevalence of hypertension (% of adults ages 30-79)"
][['Country Name'] + year_columns]

# Change the column names to the year values
filtered_adult_hypertension = filtered_adult_hypertension.rename(columns=new_col_names)
filtered_adult_hypertension = filtered_adult_hypertension.rename(columns={'Country Name': 'Country'})

# Change only year columns names to numeric 
year_columns_numeric = list(new_col_names.values())
filtered_adult_hypertension[year_columns_numeric] = filtered_adult_hypertension[year_columns_numeric].apply(pd.to_numeric, errors='coerce')

# Convert the DataFrame to a long format
filtered_adult_hypertension = filtered_adult_hypertension.melt(id_vars='Country', var_name='Year', value_name='Hypertension Prevalence (%)')

# Plot the data 
fig = px.scatter(filtered_adult_hypertension, x='Year', y='Hypertension Prevalence (%)', color='Country', title='Hypertension Prevalence (%) by Country', template="plotly_dark")
fig.show()


## Healthcare Coverage (OECD) analysis

In [4]:
# Filter data to include only Reference area, time period, and observation value, rename columns, convert time period to numeric
oecd_healthcare_filtered = (
    oecd_healthcare[
        (oecd_healthcare['Unit of measure'] == 'Percentage of population') & 
        (oecd_healthcare['Insurance type'] == 'Public and primary voluntary health insurance')
    ]
    .loc[:,['Reference area','TIME_PERIOD','OBS_VALUE']]
    .rename(columns={'Reference area': 'Country', 'TIME_PERIOD': 'Year', 'OBS_VALUE': 'Healthcare Coverage (%)'})
)
oecd_healthcare_filtered['Year'] = pd.to_numeric(oecd_healthcare_filtered['Year'])
oecd_healthcare_filtered

,Country,Year,Healthcare Coverage (%)
0,Netherlands,1960,71.0
1,Netherlands,1961,71.0
2,Netherlands,1962,71.0
3,Netherlands,1963,71.0
4,Netherlands,1964,71.0
...,...,...,...
11324,Bulgaria,2019,88.6
11325,Bulgaria,2020,88.0
11326,Bulgaria,2021,88.2
11327,Bulgaria,2022,93.5


In [5]:
# plot data
fig = px.scatter(oecd_healthcare_filtered,
                 x='Year',
                 y='Healthcare Coverage (%)',
                 color='Country',
                 title='Healthcare Coverage (%) by Country',
                 template="plotly_dark")
fig.show()

## Healthcare Coverage vs Hypertension 

In [6]:
# Merge the two dataframes on the 'Country' and 'Year' columns
merged_data = pd.merge(filtered_adult_hypertension, oecd_healthcare_filtered, on=['Country', 'Year']).drop_duplicates()

In [7]:
# plot the merged data to show the relationship between hypertension prevalence and healthcare coverage
fig = px.scatter(merged_data,
                 y='Hypertension Prevalence (%)',
                 x='Healthcare Coverage (%)',
                 color='Country',
                 title='Hypertension Prevalence (%) vs Healthcare Coverage (%) by Country',
                 hover_data=['Year', 'Country', 'Hypertension Prevalence (%)', 'Healthcare Coverage (%)'],
                 template="plotly_dark")

# toggle the visibility of the traces to off by default
for trace in fig.data:
    trace.visible = 'legendonly'

fig.show()

In [8]:
# create insurance dictionary
keys = oecd_healthcare['Insurance type'].unique()
values = [
    "- **Public Health Insurance**: This is government-run health insurance, typically funded through taxes or mandatory contributions. It provides basic health coverage to all citizens or residents, often as part of a universal healthcare system. **Primary Voluntary Health Insurance**: This refers to private health insurance that individuals or employers purchase voluntarily to cover primary healthcare needs. It may complement or replace public health insurance, depending on the country's system.",
    "- This is a comprehensive private health insurance plan that covers a wide range of healthcare services, including primary, secondary, and sometimes tertiary care. It is entirely optional and purchased by individuals or employers.",
    "- This is a type of private health insurance that focuses on covering primary healthcare services, such as general practitioner visits, basic diagnostics, and preventive care. It is often used to supplement public health insurance or fill gaps in coverage.",
    "- This type of insurance provides coverage for the same services already covered by public or other primary insurance. It is often purchased by individuals who want faster access to care, more provider options, or additional benefits (e.g., private hospital rooms).",
    "- Complementary insurance covers services that are not included in public or primary insurance plans. For example, it might cover co-payments, deductibles, or treatments excluded from the basic plan (e.g., dental, vision, or alternative medicine).",
    "- Supplementary insurance provides additional benefits beyond what is offered by public or primary insurance. This could include access to private hospitals, specialized treatments, or enhanced coverage for specific conditions.",
    "- This is a mandatory health insurance system funded and regulated by the government. It is typically financed through payroll taxes or contributions and provides basic health coverage to all eligible individuals. Examples include Medicare in Australia or the National Health Service (NHS) in the UK."
]
insurance_dict = dict(zip(keys, values))


### Key Differences:

- **Public/Government Insurance**: Mandatory, funded by taxes, and provides basic coverage.
- **Voluntary Insurance**: Optional, privately purchased, and can be primary, complementary, supplementary, or duplicate.
- **Primary Insurance**: Covers basic healthcare needs.
- **Complementary Insurance**: Fills gaps in public/primary insurance (e.g., co-pays or excluded services).
- **Supplementary Insurance**: Adds extra benefits (e.g., private hospital access).
- **Duplicate Insurance**: Covers the same services as public/primary insurance but offers additional perks.


## Overweight Adults Analysis

In [9]:
# Filter the DataFrame for the specific Series Name and select the year columns
year_columns = [f'{year} [YR{year}]' for year in range(1975, 2017)] # Generate the list of column names for the years 1990 to 2023
new_col_names = {col: int(col.split(' ')[0]) for col in year_columns} # Create a dictionary to rename the columns

overweight_adults_filtered = overweight_adults[overweight_adults['Series Name'] == "Prevalence of overweight (% of adults)"][['Country Name'] + year_columns]

# Filter the DataFrame for the specific Series Name and select the year columns
overweight_adults_filtered = overweight_adults[
    overweight_adults['Series Name'] == "Prevalence of overweight (% of adults)"][['Country Name'] + year_columns]

# Change the column names to the year values
overweight_adults_filtered = overweight_adults_filtered.rename(columns=new_col_names)
overweight_adults_filtered = overweight_adults_filtered.rename(columns={'Country Name': 'Country'})

# Change only year columns names to numeric 
year_columns_numeric = list(new_col_names.values())
overweight_adults_filtered[year_columns_numeric] = overweight_adults_filtered[year_columns_numeric].apply(pd.to_numeric, errors='coerce')

# Convert the DataFrame to a long format
overweight_adults_filtered = overweight_adults_filtered.melt(id_vars='Country', var_name='Year', value_name='Overweight Prevalence (%)')

# Plot the data 
fig = px.scatter(overweight_adults_filtered, x='Year', y='Overweight Prevalence (%)', color='Country', title='Overweight Prevalence (%) by Country', template="plotly_dark")

# Set default visibility to off
for trace in fig.data:
    trace.visible = 'legendonly'

fig.show()

# add trendline
fig = px.scatter(overweight_adults_filtered, x='Year', y='Overweight Prevalence (%)', color='Country', title='Overweight Prevalence (%) by Country', template="plotly_dark", trendline='ols')

# Set default visibility to off
for trace in fig.data:
    trace.visible = 'legendonly'

fig.show()

In [10]:
# merge with merged_data to include overweight prevalence
merged_data = pd.merge(merged_data, overweight_adults_filtered, on=['Country', 'Year']).drop_duplicates()
merged_data


,Country,Year,Hypertension Prevalence (%),Healthcare Coverage (%),Overweight Prevalence (%)
0,Australia,1990,34.2,100.0,50.3
1,Austria,1990,44.0,99.0,42.4
2,Belgium,1990,36.9,97.3,50.4
3,Canada,1990,33.0,100.0,49.1
4,Chile,1990,36.6,73.1,48.9
...,...,...,...,...,...
789,Slovenia,2016,45.4,100.0,56.1
790,Sweden,2016,32.1,100.0,56.4
791,Switzerland,2016,23.4,100.0,54.3
792,United Kingdom,2016,27.3,100.0,63.7


In [13]:
# plot the merged data to show the change in overweight prevalence over time
fig = px.scatter(merged_data,
                 y='Overweight Prevalence (%)',
                 x='Year',
                 color='Country',
                 title='Overweight Prevalence (%)Over Time',
                 hover_data=['Year', 'Country', 'Healthcare Coverage (%)', 'Overweight Prevalence (%)'],
                 template="plotly_dark")
# Set default visibility to off
for trace in fig.data:
    trace.visible = 'legendonly'
    
fig.show()

# plot the merged data to show the relationship between healthcare coverage and overweight prevalence
fig = px.scatter(merged_data,
                 y='Overweight Prevalence (%)',
                 x='Healthcare Coverage (%)',
                 color='Country',
                 title='Overweight Prevalence (%) vs Healthcare Coverage (%) by Country',
                 hover_data=['Year', 'Country', 'Healthcare Coverage (%)', 'Overweight Prevalence (%)'],
                 template="plotly_dark")
# Set default visibility to off
for trace in fig.data:
    trace.visible = 'legendonly'
    
fig.show()

- **At a glance it looks like every country increases in overweight prevalence each year, regardless of Healthcare coverage**

## Diabetes analysis

In [27]:
display(ihme_heart_failure_prevalence)
display(ihme_heart_failure_number)


,measure,location,sex,age,cause,rei,metric,year,val,upper,lower
0,Prevalence,Cambodia,Male,All ages,All causes,Heart failure,Percent,1990,0.002522,0.002944,0.002154
1,Prevalence,Cambodia,Female,All ages,All causes,Heart failure,Percent,1990,0.002329,0.002738,0.001967
2,Prevalence,Cambodia,Male,All ages,All causes,Heart failure,Percent,1991,0.002518,0.002927,0.002157
3,Prevalence,Cambodia,Female,All ages,All causes,Heart failure,Percent,1991,0.002336,0.002739,0.001979
4,Prevalence,Cambodia,Male,All ages,All causes,Heart failure,Percent,1992,0.002515,0.002915,0.002164
...,...,...,...,...,...,...,...,...,...,...,...
13051,Prevalence,Sudan,Female,All ages,All causes,Heart failure,Percent,2019,0.003287,0.003796,0.002828
13052,Prevalence,Sudan,Male,All ages,All causes,Heart failure,Percent,2020,0.004918,0.005724,0.004223
13053,Prevalence,Sudan,Female,All ages,All causes,Heart failure,Percent,2020,0.003289,0.003771,0.002840
13054,Prevalence,Sudan,Male,All ages,All causes,Heart failure,Percent,2021,0.004933,0.005690,0.004219


,measure,location,sex,age,cause,rei,metric,year,val,upper,lower
0,Prevalence,Kingdom of Cambodia,Male,All ages,All causes,Heart failure,Number,1990,12206.483871,14262.916043,10401.433878
1,Prevalence,Kingdom of Cambodia,Female,All ages,All causes,Heart failure,Number,1990,12499.323351,14707.986675,10555.153033
2,Prevalence,Kingdom of Cambodia,Male,All ages,All causes,Heart failure,Number,1991,12545.808970,14590.678476,10716.126451
3,Prevalence,Kingdom of Cambodia,Female,All ages,All causes,Heart failure,Number,1991,12839.880161,15062.020386,10870.131505
4,Prevalence,Kingdom of Cambodia,Male,All ages,All causes,Heart failure,Number,1992,12888.157947,14952.095141,11069.546545
...,...,...,...,...,...,...,...,...,...,...,...
13051,Prevalence,Republic of Sudan,Female,All ages,All causes,Heart failure,Number,2019,65431.038173,75372.092353,56321.137297
13052,Prevalence,Republic of Sudan,Male,All ages,All causes,Heart failure,Number,2020,101283.228110,117197.072987,87043.599216
13053,Prevalence,Republic of Sudan,Female,All ages,All causes,Heart failure,Number,2020,66783.076792,76585.006058,57741.425279
13054,Prevalence,Republic of Sudan,Male,All ages,All causes,Heart failure,Number,2021,103037.231167,118601.400313,88035.924746


In [31]:
ihme_heart_failure_prevalence[ihme_heart_failure_prevalence['location'] == 'United States of America']


,measure,location,sex,age,cause,rei,metric,year,val,upper,lower
5440,Prevalence,United States of America,Male,All ages,All causes,Heart failure,Percent,1990,0.011145,0.012774,0.009784
5441,Prevalence,United States of America,Female,All ages,All causes,Heart failure,Percent,1990,0.010724,0.012392,0.009429
5442,Prevalence,United States of America,Male,All ages,All causes,Heart failure,Percent,1991,0.011296,0.012955,0.009933
5443,Prevalence,United States of America,Female,All ages,All causes,Heart failure,Percent,1991,0.010852,0.012536,0.009545
5444,Prevalence,United States of America,Male,All ages,All causes,Heart failure,Percent,1992,0.011446,0.013148,0.010083
...,...,...,...,...,...,...,...,...,...,...,...
5499,Prevalence,United States of America,Female,All ages,All causes,Heart failure,Percent,2019,0.013394,0.014273,0.012555
5500,Prevalence,United States of America,Male,All ages,All causes,Heart failure,Percent,2020,0.015116,0.016127,0.014110
5501,Prevalence,United States of America,Female,All ages,All causes,Heart failure,Percent,2020,0.013597,0.014543,0.012723
5502,Prevalence,United States of America,Male,All ages,All causes,Heart failure,Percent,2021,0.015418,0.016575,0.014337
